# Implementing a File System

We will explore how to build a Very Simple File System (VSFS). The original file system for UNIX was very similar to VSFS.

The OS views the hard disk as a set of N 4KB blocks that it read and write from/to.

<br>
<img src="images/02-blocks.png" width="500">
<br>

## How do we build a file system?

What do we need to store?

1) All of the user's data, stored in files
    - stored in **data blocks**
2) Per-file meta data: creation/modification time, permissions, size in bytes, # blocks required per file, ptrs to the blocks. 
    - Metadata is stored in **inodes**. inodes are stored in **inodes blocks**.
3) Directories! Directories are just files. They contain a list of all files and directories within them.
4) A way to track which inode and data blocks are free.
    - This is done using two bitmaps, the **inode blocks bitmap** and the **data blocks bitmap**
5) Meta data for the entire file system: start/size of the data region, inodes region, bitmap locations, file system type, etc.
    - Stored in the **super block**

<br>
<img src="images/04-blocks.png" width="500">
<br>

Whenever we want to create, read, or write to/from files, we will need to access and potentially update inodes, data blocks, and (sometime) bitmaps.

# Example Operations

## Creating some file: /foo

In order to create `/foo`, we need to update the ROOT so that it lists `foo` and we need to allocate an inode for this new file.

Steps:
1) read in the ROOT directory
    - READ in the inode
    - READ in its data block
    - WRITE out the inode (update access time)
2) create `/foo`
    - READ in the inode bitmaps, find a free spot
    - WRITE out the updated bitmap
    - WRITE out the inode for `/foo`
3) update the ROOT directory
    - WRITE out ROOT's data block to include `foo`
    - WRITE out the updated inode: modification time, size, etc.

## Read a file: /foo/bar

1) read in the ROOT directory
    - READ in the inode
    - READ in its data block to find out `foo`'s inode number
    - WRITE out the inode (update access time)
2) read in `foo`
    - READ in its inode
    - READ in its data block to find out `bar`'sw inode number
    - WRITE out the inode (update access time)
3) read in `bar`
    - READ in its inode
    - READ in its data block(s)
    - WRITE out the inode (update access time)



<br>
<img src="images/05-read-foo-bar.png" width="500">
<br>

## Write to file: /foo/bar

<br>
<img src="images/06-write-foo-bar.png" width="500">
<br>